# Load a Pandas DataFrame to a temporary table in a SQL databse

Sometimes I want to join data that can easily be loaded from a CSV file into a Pandas DataFrame with data in a SQL database (that is too big to easily work with in a DataFrame). For this case, I'll leverage the database to do the join. To get the DataFrame into the SQL database, I want to create a temporary table. However, Pandas doesn't support creating temporary tables without a little bit of extra help.

In [1]:
# Setup

import os

from IPython.display import display
import pandas as pd

# Define some paths where we will store the database.
NOTEBOOK_DIR = os.getcwd()
DATA_DIR = os.path.abspath(os.path.join(NOTEBOOK_DIR, os.pardir, 'data'))
DATA_DIR_TMP = os.path.join(DATA_DIR, 'tmp')

os.makedirs(DATA_DIR_TMP, exist_ok=True)

# Define the path and the SQLAlchemy database URL to connect to the database.

DB_PATH = os.path.join(DATA_DIR_TMP, 'example.db')
# Note that for SQLite databases with an absolute path, you need three
# slashes after the `sqlite:`, so that's four slashes in total when
# you count the leading `/` of the path.
DB_URL = f"sqlite:///{DB_PATH}"

## Creating a class to represent a temporary table

Pandas has a `SQLTable` class to represent a table in a SQL database. We have to create a subclass that adds the `TEMPORARY` prefix when executing the `CREATE TABLE` query.

This method comes from this gist: [pandas_temporary_table.py](https://gist.github.com/alecxe/44682f79b18f0c82a99c).

In [2]:
from pandas.core.dtypes.common import is_list_like
from pandas.io.sql import SQLTable


class TemporarySQLTable(SQLTable):
    """Temporary table in a SQL database."""
    # HACK: Override this private method to create a temporary table.
    # This method is via https://gist.github.com/alecxe/44682f79b18f0c82a99c
    def _create_table_setup(self):
        """Create a SQL Alchemy Table object."""
        from sqlalchemy import Table, Column, PrimaryKeyConstraint

        column_names_and_types = self._get_column_names_and_types(
            self._sqlalchemy_type
        )

        columns = [Column(name, typ, index=is_index)
                   for name, typ, is_index in column_names_and_types]

        if self.keys is not None:
            if not is_list_like(self.keys):
                keys = [self.keys]
            else:
                keys = self.keys
            pkc = PrimaryKeyConstraint(*keys, name=self.name + '_pk')
            columns.append(pkc)

        schema = self.schema or self.pd_sql.meta.schema

        # At this point, attach to new metadata, only attach to self.meta
        # once table is created.
        from sqlalchemy.schema import MetaData
        meta = MetaData(self.pd_sql, schema=schema)

        # Adding the `TEMPORARY` prefix here is what makes the table temporary.
        prefixes = ['TEMPORARY']
        return Table(self.name, meta, *columns, schema=schema, prefixes=prefixes)

    def _execute_create(self):
        # Inserting table into database, add to MetaData object
        self.table = self.table.tometadata(self.pd_sql.meta)

        # allow creation of temporary tables
        self.table._prefixes.append('TEMPORARY')

        self.table.create()


## Create some toy data

In [3]:
new_alderpeople_data = [
    (1, 'Daniel La Spata'),
    (16, 'Stephanie Coleman'),    
    (20, 'Jeannette Taylor'),
    (22, 'Michael Rodriguez'),
    (25, 'Byron Sigcho-Lopez'),
    (31, 'Felix Cardona Jr.'),
    (33, 'Rossana Rodriguez Sanchez'),
    (39, 'Sam Nugent'),
    (40, 'Andre Vasquez'),
    (45, 'Jim Gardiner'),
    (47, 'Matt Martin'),
    (49, 'Maria Hadden'),
]
columns = ['ward', 'name']

new_alderpeople = pd.DataFrame.from_records(
    new_alderpeople_data,
    columns=columns
)

new_alderpeople

,ward,name
0,1,Daniel La Spata
1,16,Stephanie Coleman
2,20,Jeannette Taylor
3,22,Michael Rodriguez
4,25,Byron Sigcho-Lopez
5,31,Felix Cardona Jr.
6,33,Rossana Rodriguez Sanchez
7,39,Sam Nugent
8,40,Andre Vasquez
9,45,Jim Gardiner


## Connect to the database and load the data into a temporary table

In [4]:
from pandas.io.sql import pandasSQL_builder
from sqlalchemy import create_engine

engine = create_engine(DB_URL)

with engine.connect() as conn, conn.begin():
    pandas_engine = pandasSQL_builder(conn)
    table = TemporarySQLTable(
        'alderpeople',
        pandas_engine,
        frame=new_alderpeople,
        if_exists='replace'
    )
    table.create()

    new_alderpeople.to_sql(
        'alderpeople',
        conn,
        if_exists='append'
    )
    
    # Check that it worked.
    sql_query = """
        SELECT * FROM alderpeople;
    """
    display(pd.read_sql(sql_query, con=conn))

,index,ward,name
0,0,1,Daniel La Spata
1,1,16,Stephanie Coleman
2,2,20,Jeannette Taylor
3,3,22,Michael Rodriguez
4,4,25,Byron Sigcho-Lopez
5,5,31,Felix Cardona Jr.
6,6,33,Rossana Rodriguez Sanchez
7,7,39,Sam Nugent
8,8,40,Andre Vasquez
9,9,45,Jim Gardiner
